In [259]:
# utilities
import re
import pickle
import numpy as np
import pandas as pd
import nltk

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *

# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
import snowballstemmer

import ast
from io import StringIO

In [260]:
#import dataset to juypter notebook and store it into a pandas dataframe:
twitterdataset = pd.read_csv('twitter.csv',encoding='latin-1')
twitterdataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [261]:
#now we need to add columns to our dataset, as seen above there are no column names:
twitterdataset.columns = ["target", "id", "date", "flag", "user", "text"]

In [262]:
#now print out dataset:
twitterdataset.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [263]:
#drop the columns we do not need, we only need target and the actual text of the 
twitterdataset.drop(columns=['id', 'date', 'flag', 'user' ])

,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew
...,...,...
1599994,4,Just woke up. Having no school is the best fee...
1599995,4,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,Happy 38th Birthday to my boo of alll time!!! ...


In [264]:
#turn it into a binary problem, we only want postive or negative tweets. So that is if the value is 4 or 0
find_postive_data = twitterdataset[twitterdataset.target==4]
find_negative_data = twitterdataset[twitterdataset.target==0]

find_postive_data.drop(columns=['id', 'date', 'flag', 'user' ], axis = 1, inplace= True)
find_negative_data.drop(columns=['id', 'date', 'flag', 'user' ],axis = 1, inplace= True)




/var/folders/6k/htbjx7l93ldb6p54pmscffxh0000gn/T/ipykernel_50215/1520854249.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  find_postive_data.drop(columns=['id', 'date', 'flag', 'user' ], axis = 1, inplace= True)
/var/folders/6k/htbjx7l93ldb6p54pmscffxh0000gn/T/ipykernel_50215/1520854249.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  find_negative_data.drop(columns=['id', 'date', 'flag', 'user' ],axis = 1, inplace= True)


In [265]:
find_postive_data.head()

,target,text
799999,4,I LOVE @Health4UandPets u guys r the best!!
800000,4,im meeting up with one of my besties tonight! ...
800001,4,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800002,4,Being sick can be really cheap when it hurts t...
800003,4,@LovesBrooklyn2 he has that effect on everyone


In [266]:
find_negative_data.head()

,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [267]:
#now we want to put the negative and positive tweets together:
add_set = [find_postive_data,find_negative_data]
binary_twitter_set = pd.concat(add_set)

In [268]:
binary_twitter_set.tail()

,target,text
799994,0,Sick Spending my day laying in bed listening ...
799995,0,Gmail is down?
799996,0,rest in peace Farrah! So sad
799997,0,@Eric_Urbane Sounds like a rival is flagging y...
799998,0,has to resit exams over summer... wishes he w...


In [269]:
#first lets convert dataset to lowercase, the text portion of the dataset:
binary_twitter_set['text'] = binary_twitter_set['text'].str.lower()
binary_twitter_set.tail()


,target,text
799994,0,sick spending my day laying in bed listening ...
799995,0,gmail is down?
799996,0,rest in peace farrah! so sad
799997,0,@eric_urbane sounds like a rival is flagging y...
799998,0,has to resit exams over summer... wishes he w...


In [270]:
#cleaning stop words:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kartiksharma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kartiksharma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [271]:
#cleaning stop words:
stopwords = set(stopwords.words('english'))
def clean_stopwords(text):
    returnText = ''
    for word in str(text).split():
        if(word not in stopwords):
            returnText = returnText +" "+ word
    
    return returnText


binary_twitter_set['text'] = binary_twitter_set['text'].apply(lambda x: clean_stopwords(x))

In [272]:
binary_twitter_set

,target,text
799999,4,love @health4uandpets u guys r best!!
800000,4,im meeting one besties tonight! cant wait!! -...
800001,4,"@darealsunisakim thanks twitter add, sunisa! ..."
800002,4,sick really cheap hurts much eat real food pl...
800003,4,@lovesbrooklyn2 effect everyone
...,...,...
799994,0,sick spending day laying bed listening @taylo...
799995,0,gmail down?
799996,0,rest peace farrah! sad
799997,0,@eric_urbane sounds like rival flagging ads. ...


In [273]:
#cleaning usernames:

def remove_user_names(text):
    #USER RE.SUB(pat, replacement, str) function searches for all the instances of pattern in the given string, and replaces them
    subsitution = re.sub('@[^\s]+'," ",text) #REGEX from stackoverflow
    return subsitution

binary_twitter_set['text'] = binary_twitter_set['text'].apply(lambda text: remove_user_names(text))

In [274]:
binary_twitter_set

,target,text
799999,4,love u guys r best!!
800000,4,im meeting one besties tonight! cant wait!! -...
800001,4,"thanks twitter add, sunisa! got meet hin sh..."
800002,4,sick really cheap hurts much eat real food pl...
800003,4,effect everyone
...,...,...
799994,0,sick spending day laying bed listening
799995,0,gmail down?
799996,0,rest peace farrah! sad
799997,0,sounds like rival flagging ads. much though


In [275]:
#removing urls from tweets:
def remove_url(text):
    #USER RE.SUB(pat, replacement, str) function searches for all the instances of pattern in the given string, and replaces them
    subsitution = re.sub('r"((https|http|ftp)?(:\/\/)?(www\.)?)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&\/\/=]*)"'," ",text) #REGEX from stackoverflow
    return subsitution

binary_twitter_set['text'] = binary_twitter_set['text'].apply(lambda text: remove_url(text))

In [276]:
def remove_url_again(text):
    #USER RE.SUB(pat, replacement, str) function searches for all the instances of pattern in the given string, and replaces them
    subsitution = re.sub(r"http\S+","",text) #REGEX from stackoverflow
    return subsitution

binary_twitter_set['text'] = binary_twitter_set['text'].apply(lambda text: remove_url_again(text))

In [277]:
#remove anything that is not an alphabet:
def remove_non_alphabet(text):
    #USER RE.SUB(pat, replacement, str) function searches for all the instances of pattern in the given string, and replaces them
    subsitution = re.sub("[^a-zA-Z0-9]"," ",text) #REGEX from stackoverflow
    return subsitution

binary_twitter_set['text'] = binary_twitter_set['text'].apply(lambda text: remove_non_alphabet(text))

In [278]:
#We need tokenize first:

binary_twitter_set['text'] = binary_twitter_set['text'].apply(lambda text: word_tokenize(text))


In [279]:
binary_twitter_set

,target,text
799999,4,"[love, u, guys, r, best]"
800000,4,"[im, meeting, one, besties, tonight, cant, wai..."
800001,4,"[thanks, twitter, add, sunisa, got, meet, hin,..."
800002,4,"[sick, really, cheap, hurts, much, eat, real, ..."
800003,4,"[effect, everyone]"
...,...,...
799994,0,"[sick, spending, day, laying, bed, listening]"
799995,0,"[gmail, down]"
799996,0,"[rest, peace, farrah, sad]"
799997,0,"[sounds, like, rival, flagging, ads, much, tho..."


In [280]:
#stemming:
def stemming(text):
    returnArray= []
    for word in text:
        returnArray.append( nltk.stem.snowball.SnowballStemmer(language='english').stem(word))
    return returnArray

binary_twitter_set['text'] = binary_twitter_set['text'].apply(lambda text: stemming(text))


In [281]:
binary_twitter_set

,target,text
799999,4,"[love, u, guy, r, best]"
800000,4,"[im, meet, one, besti, tonight, cant, wait, gi..."
800001,4,"[thank, twitter, add, sunisa, got, meet, hin, ..."
800002,4,"[sick, realli, cheap, hurt, much, eat, real, f..."
800003,4,"[effect, everyon]"
...,...,...
799994,0,"[sick, spend, day, lay, bed, listen]"
799995,0,"[gmail, down]"
799996,0,"[rest, peac, farrah, sad]"
799997,0,"[sound, like, rival, flag, ad, much, though]"


In [282]:
def lemmer(text):
    lemmi = WordNetLemmatizer()
    returnArray= []
    for word in text:
        returnArray.append(lemmi.lemmatize(word))
    return returnArray


binary_twitter_set['text'] = binary_twitter_set['text'].apply(lambda text: lemmer(text))

In [283]:
#turn the array of words back in to sentences:
binary_twitter_set['text'] = binary_twitter_set['text'].apply(lambda text: ' '.join(text))
binary_twitter_set

,target,text
799999,4,love u guy r best
800000,4,im meet one besti tonight cant wait girl talk
800001,4,thank twitter add sunisa got meet hin show dc ...
800002,4,sick realli cheap hurt much eat real food plus...
800003,4,effect everyon
...,...,...
799994,0,sick spend day lay bed listen
799995,0,gmail down
799996,0,rest peac farrah sad
799997,0,sound like rival flag ad much though


In [284]:
#set our y and X values of the pandas dataframe now that we are done cleaning the dataset:
y_labels = binary_twitter_set.target
X_text = binary_twitter_set.text

In [285]:
X_text

799999                                    love u guy r best
800000        im meet one besti tonight cant wait girl talk
800001    thank twitter add sunisa got meet hin show dc ...
800002    sick realli cheap hurt much eat real food plus...
800003                                       effect everyon
                                ...                        
799994                        sick spend day lay bed listen
799995                                           gmail down
799996                                 rest peac farrah sad
799997                 sound like rival flag ad much though
799998    resit exam summer wish work harder first year uni
Name: text, Length: 1599999, dtype: object

In [289]:
X_train, X_test, y_train, y_test = train_test_split(X_text,y_labels,test_size = 0.05, random_state =42)

In [290]:
X_train

1304327               ur name fabul simpli follow u caus lol
996808                                 would bring breakfast
987558     ok it real twitterbreak tweet later pahinga mu...
1446525    oh good i sit around watch charm dvd sudden ep...
628076                                               go home
                                 ...                        
1059177    yay atwin babi boy matthew john last night 8 l...
614414     i ve realis much im gon na miss phil biolog te...
931931                                       like meeeeeeeee
1471154                                           okay sleep
921957     sleep well wk gig fri nite amp woke earli sat ...
Name: text, Length: 1519999, dtype: object